In [1]:
import torch
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
from peft import PeftModel

bin C:\Users\F4LC0n\AppData\Local\Programs\Python\Python310\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


In [2]:
base_model_id = "mistralai/Mistral-7B-v0.1"
model_max_length = 512
project = "alpaca-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

In [3]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True)

In [4]:
# model
bnb = BitsAndBytesConfig(load_in_4bit=True,
                         bnb_4bit_use_double_quant=True,
                         bnb_4bit_quant_type="nf4",
                         bnb_4bit_compute_dtype=torch.bfloat16)

base_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb)
model =PeftModel.from_pretrained(base_model, 'mistral-alpaca-finetune/checkpoint-1000')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# evaluation inputs
eval_input = {'instruction': 'Construct a metaphor that compares life to a road.'}
eval_prompt = ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n").format_map(eval_input)

In [6]:
# eval
input = tokenizer(eval_prompt, return_tensors="pt").to('cuda')
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**input, max_new_tokens=model_max_length)[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Construct a metaphor that compares life to a road.

### Response:
Life is like a road, with twists and turns, ups and downs, and unexpected detours. Just as a road leads to different destinations, life presents us with choices and opportunities to take different paths. Sometimes, we may feel lost or unsure of where we are going, but with perseverance and determination, we can find our way and reach our destination. Along the way, we may encounter obstacles and challenges, but with the right tools and support, we can overcome them and keep moving forward. Ultimately, the journey of life is about finding our purpose, making the most of our time, and enjoying the ride. Just as a road leads to new adventures and experiences, life is a journey filled with opportunities to grow, learn, and discover. So, let's hit the road and make the most of every mile.</s>
